# DEMO: JOB & Modelo Relacional

# 1 JOB

## 1.1 Creación de Jerarquía

> Copia esta celda de codigo y pegalo en una celda de un notebook =>

```
%sql
CREATE CATALOG IF NOT EXISTS sesion4;

-- 01 · Landing / Raw
CREATE SCHEMA IF NOT EXISTS sesion4.layer10_landing;
CREATE VOLUME IF NOT EXISTS sesion4.layer10_landing.volume;

-- 02 · Bronze
CREATE SCHEMA IF NOT EXISTS sesion4.layer20_bronze;

-- 03 · Silver
CREATE SCHEMA IF NOT EXISTS sesion4.layer30_silver;
CREATE SCHEMA IF NOT EXISTS sesion4.layer30_silver_quarantine;

-- 04 · Gold
CREATE SCHEMA IF NOT EXISTS sesion4.layer40_gold;

-- 99 · Operaciones / Configuración
CREATE SCHEMA IF NOT EXISTS sesion4.layer99_ops;
CREATE VOLUME IF NOT EXISTS sesion4.layer99_ops.configs;
```

## 1.2 Movemos la información desde el Repo

> Copia esta celda de codigo y pegalo en una celda de un notebook =>

```
%sh
curl -L https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/refs/heads/main/sesion3/data/dim_aircraft.csv -o /Volumes/sesion4/layer10_landing/volume/dim_aircraft.csv

curl -L https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/refs/heads/main/sesion3/data/dim_phase.csv -o /Volumes/sesion4/layer10_landing/volume/dim_phase.csv

curl -L https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/refs/heads/main/sesion3/data/dim_sensor_reading.csv -o /Volumes/sesion4/layer10_landing/volume/dim_sensor_reading.csv

curl -L https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/refs/heads/main/sesion3/data/fact_sensor.csv -o /Volumes/sesion4/layer10_landing/volume/fact_sensor.csv

curl -L https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/refs/heads/main/sesion3/data/schemas_enriched_pkfk.json -o /Volumes/sesion4/layer99_ops/configs/schemas_enriched_pkfk.json
```

## 1.3 Creacion del Workflow

### 1.3.1 Creacion del JOB I

> * Step1: Dirigite en el Menu izquierdo a el apartado "Jobs & Pipelines" haz click
> * Step2: Haz click en el boton azul de la parte derecha "Create"
> * Step3: Haz click en la opcion Job

<img src="https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/main/photos/sesion2_createjob1.png" width="1600px"/>

### 1.3.2 Creacion del JOB II

Tras crear el JOB aparecera esta ventana. Aqui hay informacion diversa sobre el workflow que iremos creando.

Vamos a empezar creando el primer JOB. 

> * Simplemente haz click en "+Add annother task type"

<img src="https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/main/photos/sesion2_createjob2.png" width="1600px"/>

### 1.3.3 Definir el JOB

Al crear el JOB aparecerá la siguiente pantalla

Ahora vamos a completar los campos vacíos Task name, type, Source....

💡 *Consejo: Utiliza nombres de tarea claros y estructurados, especialmente cuando diseñes workflows con varias tareas conectadas.*

<img src="https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/main/photos/sesion2_createjob3.png" width="1600px"/>

#### Definamos y completemos la actividad que va a realizar el JOB

* Task name = ingest_dim_aircraft
* Type = python script
* path = navega en busca del script 01_ingesta_Datos.py

<img src="https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/main/photos/jobnew1.png" width="600px"/>

<img src="https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/main/photos/jobnew2.png" width="600px"/>

* Compute = Serverless
* Environment and Libraries = Default
* Parameters = ["--input_file=/Volumes/sesion4/layer10_landing/volume/dim_aircraft.csv","--output_table=sesion4.layer20_bronze.dim_aircraft","--write_mode=overwrite","--partition_by="]


Finalmente dale al botón de abajo a la derecha "Create Task"

### 3.4 Crear una segunda Tarea

Vamos a añadir una **segunda tarea** dentro de nuestro Job.  
Para ello, simplemente haz clic en **“+ Add task”** y selecciona en nuestro caso la opción **“Python script”**.

Esto nos permitirá incorporar un script Python como nueva tarea dentro del workflow y posteriormente configurar su **dependencia** respecto a la tarea inicial.


##### 3.4.1 Selecciona "Add Task2"

<img src="https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/main/photos/jobnew3.png" width="600px"/>

##### 3.4.2 Selecciona Python Script como tipo de tarea

<img src="https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/main/photos/jobnew4.png" width="1000px"/>

### 3.5 Creacion Segunda Tarea

* Task name = Validacion_aircraftV2
* Type = python script
* path = navega en busca del script 02_validar_datos_sol.py


<img src="https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/main/photos/jobnew5.png" width="600px"/>


* Compute = Serverless
* Environment and Libraries = Default
* Parameters = ["--input_table=sesion4.layer20_bronze.dim_aircraft","--output_table=sesion4.layer30_silver.dim_aircraft","--config_path=/Volumes/sesion4/layer99_ops/configs/schemas_enriched_pkfk.json","--table_name=dim_aircraft","--write_mode=overwrite","--quarantine_table=sesion4.layer30_silver_quarantine.dim_aircraft_quarantine"]
* Finalmente dale al botón de abajo a la derecha "Create Task"

### 3.6 Ejecutar el JOB

Para **ejecutar el Job completo**, incluyendo todas sus tareas encadenadas, basta con hacer clic en el botón **“Run now”** en la parte superior derecha.

En nuestro caso, esto lanzará la ejecución secuencial de:

1. `01_ingesta_Datos.py`  
2. `02_validar_datos.py` (dependiente del éxito de la primera)




##### 🔍 Ejecución individual de tareas
Además, si deseas ejecutar una tarea específica de forma independiente, puedes usar el **icono de ▶️ (play)** disponible en cada Task.  
Esto es útil para pruebas o depuración de una parte del workflow sin necesidad de ejecutar el Job completo.

💡 *En un workflow real, es recomendable probar cada task de forma aislada antes de ejecutar toda la cadena.*


<img src="https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/main/photos/jobnew6.png" width="1200px"/>

### 3.7 Comprobar evolución de la Ejecución

Tras lanzar la ejecución del Job en el paso anterior, las tareas comenzarán a ejecutarse según el flujo configurado.  
> Para **monitorizar el progreso en tiempo real**, podemos acceder al apartado **“Runs”**.

Desde esta sección podemos:

- Visualizar el estado de cada ejecución (Running, Success, Failed, Canceled).
- Seguir el orden de ejecución y dependencias entre tareas.
- Consultar información de logs, tiempos de ejecución y posibles errores.
- Acceder directamente al detalle de cada Task para analizar su comportamiento.

💡 *“Runs” es el panel central de seguimiento y diagnóstico de tus workflows en Databricks.*


<img src="https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/main/photos/sesion2_createjob7.png" width="1600px"/>

Dentro del apartado **Runs** podrás visualizar tanto los **Jobs en ejecución** como los **Jobs ejecutados anteriormente**.

Desde aquí puedes:

- Ver el **histórico de ejecuciones** con su estado (Success, Failed, Running, Canceled).
- Acceder al **detalle de cada ejecución**, incluyendo:
  - Logs de cada tarea.
  - Tiempo de inicio y fin.
  - Duración total.
  - Cluster utilizado.
  - Estructura visual del workflow y dependencias.
- En caso de **fallo de una tarea**, consultar los **errores detallados**, el stack trace y la etapa exacta donde ocurrió el problema.



💡 *Este panel es fundamental para el análisis y depuración de workflows en Databricks.*


<img src="https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/main/photos/sesion2_createjob8.png" width="1600px"/>

#### 3.8.1 Abrir el archivo YAML

Dirígete dentro del **Workspace** a la carpeta **`src`** de este curso y abre el archivo **`workflow.yml`**.

En este archivo podrás visualizar cómo se define un workflow mediante YAML, incluyendo:

- La estructura de **tareas (tasks)**.
- Las **dependencias** entre ellas.
- La configuración del **cluster**.
- Los **parámetros**, rutas de notebooks o scripts.
- Opciones de **notificaciones**, desencadenantes (triggers), y más.

Este archivo representa la **versión declarativa** del workflow, es decir, su definición escrita y reutilizable.


> => **Copia todo el contenido**

<img src="https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/main/photos/jobnew7.png" width="900px"/>


#### 3.8.2 Editar el YAML en tu JOB

> * Dirígete al apartado **Jobs & Pipelines** y haz clic sobre tu **Job**.
> * Una vez dentro del detalle del Job, en la esquina superior derecha, haz clic en los **tres puntos (⋮)** y selecciona la opción **“Edit as YAML”**.
> * Una vez dentro del YAML, borra todo el contenido y pega el que has copiado anteriormente


💡 *“Edit as YAML” es una forma declarativa de gestionar workflows, ideal para automatización y control de versiones.*


<img src="https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/main/photos/sesion2_yaml2.png" width="1600px"/>

#### 3.8.3 Modificaciones de path

Deberás realizar las modificaciones correspondientes en las rutas (**path**) dentro del archivo YAML.

Si has seguido correctamente la estructura propuesta en este curso, será suficiente con **ajustar únicamente la parte del nombre de usuario** dentro de las rutas, manteniendo el resto tal como está.

> Haz click en Close editor una vez termines de modificar

💡 *Asegúrate de que las rutas apunten a tus notebooks o scripts dentro de tu Workspace para que el workflow pueda ejecutarlos correctamente.*


<img src="https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/main/photos/sesion2_yaml3.png" width="1600px"/>

# 2 Modelo Relacional

## 2.1 Definimos las constraints

%md
Databricks permite definir y visualizar relaciones entre tablas directamente desde Unity Catalog, utilizando constraints de tipo Primary Key (PK) y Foreign Key (FK).

Estas relaciones no solo documentan el modelo de datos, sino que además refuerzan la integridad referencial y facilitan el análisis visual de dependencias.

%md
#### ¿Por qué son importantes?
- **Documentación y semántica**: Las `PRIMARY KEY` y `FOREIGN KEY` ayudan a describir la estructura lógica del modelo de datos.
- **Optimización de consultas**: Databricks puede usar estas relaciones para mejorar los planes de ejecución.
- **Calidad de datos**: Las restricciones como `NOT NULL` y `CHECK` **sí se aplican** y evitan datos inválidos.
- **Integridad referencial (informacional)**: Aunque las PK/FK en Databricks son **informacionales** (no bloquean duplicados ni huérfanos), son útiles para gobernanza y análisis.

#### Tipos de Constraints en Databricks
- **NOT NULL**: Se valida y evita valores nulos en la columna.
- **CHECK**: Se valida y asegura reglas simples (ej. rangos, condiciones).
- **PRIMARY KEY / FOREIGN KEY**: Informacionales, no se aplican físicamente, pero aportan metadata y optimización.

#### Consideraciones
- Antes de aplicar `NOT NULL`, asegúrate de que no existan valores nulos.
- Las PK/FK **no impiden** duplicados ni huérfanos, pero son recomendadas para claridad y optimización.
- Requieren **Unity Catalog** para ser definidas.



> Copia esta celda de codigo y pegalo en una celda de un notebook =>

```
%sql
-- Evita nulos a nivel de metadatos
ALTER TABLE sesion4.layer30_silver.dim_phase
  ALTER COLUMN phase_key SET NOT NULL;

-- Crea la primary key (fallará si quedan duplicados)
ALTER TABLE sesion4.layer30_silver.dim_phase
  ADD CONSTRAINT pk_dim_phase PRIMARY KEY (phase_key);

-- Evita nulos a nivel de metadatos
ALTER TABLE sesion4.layer30_silver.dim_aircraft
  ALTER COLUMN aircraft_id SET NOT NULL;

-- Crea la primary key (fallará si quedan duplicados)
ALTER TABLE sesion4.layer30_silver.dim_aircraft
  ADD CONSTRAINT pk_aircraft_id PRIMARY KEY (aircraft_id);

  -- Evita nulos a nivel de metadatos
ALTER TABLE sesion4.layer30_silver.dim_sensor_reading
  ALTER COLUMN reading_id SET NOT NULL;

-- Crea la primary key (fallará si quedan duplicados)
ALTER TABLE sesion4.layer30_silver.dim_sensor_reading
  ADD CONSTRAINT pk_reading_id PRIMARY KEY (reading_id);

-- FACT: PK y FKs no nulas
ALTER TABLE sesion4.layer30_silver.fact_engine_sensor ALTER COLUMN reading_id       SET NOT NULL;
ALTER TABLE sesion4.layer30_silver.fact_engine_sensor ALTER COLUMN aircraft_id      SET NOT NULL;
ALTER TABLE sesion4.layer30_silver.fact_engine_sensor ALTER COLUMN phase_of_flight  SET NOT NULL;


-- FACT (definimos primary key en fact table)
ALTER TABLE sesion4.layer30_silver.fact_engine_sensor
ADD CONSTRAINT pk_fact_engine_sensor PRIMARY KEY (reading_id);


-- FK: fact.aircraft_id → dim_aircraft.aircraft_id
ALTER TABLE sesion4.layer30_silver.fact_engine_sensor
ADD CONSTRAINT fk_fact_engine_sensor_aircraft
FOREIGN KEY (aircraft_id)
REFERENCES sesion4.layer30_silver.dim_aircraft(aircraft_id);

-- FK: fact.phase_of_flight → dim_phase.phase_key
ALTER TABLE sesion4.layer30_silver.fact_engine_sensor
ADD CONSTRAINT fk_fact_engine_sensor_phase
FOREIGN KEY (phase_of_flight)
REFERENCES sesion4.layer30_silver.dim_phase(phase_key);

-- FK: fact.reading_id → dim_sensor_reading.reading_id
ALTER TABLE sesion4.layer30_silver.fact_engine_sensor
ADD CONSTRAINT fk_fact_engine_sensor_reading
FOREIGN KEY (reading_id)
REFERENCES sesion4.layer30_silver.dim_sensor_reading(reading_id);
```

## 2.2 Visualizar relacion

Una vez definidas las **claves primarias** y **claves foráneas** en las distintas tablas, podemos **visualizar el modelo relacional resultante** en Databricks.  
Este paso nos permite comprender mejor la estructura lógica del **Lakehouse**, identificar las relaciones entre dimensiones y hechos, y validar que el diseño sigue las buenas prácticas de


* Dirigite al catalog
* navega hacia la tabla sesion4.layer30_silver.fact_engine_sensor

<img src="https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/main/photos/jobnew8_1.png" width="800">

* Haz click en el botón View Relationship

Deberá entonces aparecerte la relación que acabamos de crear

<img src="https://raw.githubusercontent.com/jmartinezceste/Orquestacion_sesion2/main/photos/jobnew8.png" width="1200">